## **Web Scraping de TUCARRO.COM**

### **1. Importamos las respectivas librerías**

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

### **2. Hacemos el respectivo requerimiento a la página**

➡️ Necesitamos un indicador que nos diga si la conexion fue realmente efectiva, en este caso, **200** es el código que muestra una conexion exitosa.

🗒️ **Observación:** Para una mayor informacion, les dejo el siguiente link 🔗 https://developer.mozilla.org/es/docs/Web/HTTP/Status/200

In [12]:
html_text = requests.get('https://carros.tucarro.com.co/')
print(html_text.status_code)

200


➡️ Convertimos la pagina que vamos a scrapear en una variable para poder usarla luego como un registro de donde esta saliendo la informacion

In [13]:
conexion_url = html_text.url
print(conexion_url)

https://carros.tucarro.com.co/


### **3. Seleccionamos la grilla específica de donde vamos a sacar la información**

➡️ La informacion que necesitamos esta contenida en el siguiente elemento HTML *'ui-search-layout ui-search-layout--grid'*.

In [14]:
soup = BeautifulSoup(html_text.text, 'lxml')
cars_grill = soup.find('ol', class_ = 'ui-search-layout ui-search-layout--grid')
cars = cars_grill.find_all('li', class_ = 'ui-search-layout__item')

### **4. Extraemos la información correspondiente**

➡️ Necesitamos guardar la informacion que vamos a extraer de la página web. Para esta aplicacion, vamos a usar la memoria del sistema para almancenar los datos a traves de un dataframe.

🗒️ **Observación:** Una aplicación real requiere montar un back-end donde la información persista una vez es descargada

In [5]:
# Declaramos un datafram vacio en el cual vamos a guardar la informacion posteriormente
df = pd.DataFrame()

#### 4.1 Nombre del carro

➡️ Iniciamos a iterarar para buscar cada uno de los elementos que necesitamos encontrar.

In [28]:
#Creamos una lista vacio donde vamos a almacenar la informacion
car_name_list = []

for car in cars:
    car_name = car.find('h2', class_ = 'poly-box').text  
    car_name_list.append(car_name)

df = pd.DataFrame(car_name_list, columns=['Nombre_del_vehiculo'])
print(df.head(5))

                   Nombre_del_vehiculo
0      Toyota Fortuner 2.4l Street 4x2
1                Toyota Hilux 2.7 Wt-i
2  Mercedes-benz Clase Gla 1.3 Gla 200
3         Kia Sorento Ex 3.5 Xm At 4x4
4               Ford Escape 2.0 Se 4x2


#### 4.2 Precio del carro

In [33]:
car.find('span', class_ = 'andes-money-amount__fraction').text

'179.000.000'

In [34]:
#Creamos una lista vacio donde vamos a almacenar la informacion
price_car_list = []

for car in cars:
    price_car = car.find('span', class_ = 'andes-money-amount__fraction').text

    price_car_list.append(price_car)

df['Precio_del_vehiculo'] = price_car_list
print(df.head(5))

                   Nombre_del_vehiculo Precio_del_vehiculo
0      Toyota Fortuner 2.4l Street 4x2         179.000.000
1                Toyota Hilux 2.7 Wt-i          65.000.000
2  Mercedes-benz Clase Gla 1.3 Gla 200         170.000.000
3         Kia Sorento Ex 3.5 Xm At 4x4          59.000.000
4               Ford Escape 2.0 Se 4x2          56.500.000


#### 4.3 Model y Kilometraje del carro

In [40]:
car.find_all('li', class_ = 'poly-attributes-list__item poly-attributes-list__bar')[1].text

'29.032 Km'

In [37]:
int(car.find_all('li', class_ = 'poly-attributes-list__item poly-attributes-list__bar')[1].text)

ValueError: invalid literal for int() with base 10: '29.032 Km'

In [42]:
#Creamos una lista vacio donde vamos a almacenar la informacion
model_car_list = []
km_car_list = []

for car in cars:
    # Extraemos el modelo del carro
    model_car = int(car.find_all('li', class_ = 'poly-attributes-list__item poly-attributes-list__bar')[0].text)
    model_car_list.append(model_car)

    # Extraemos el kilometraje del carro
    km_car = car.find_all('li', class_ = 'poly-attributes-list__item poly-attributes-list__bar')[1].text
    km_car = int(km_car.replace(" Km","").replace(".",""))
    km_car_list.append(km_car)

df['Modelo_del_vehiculo'] = model_car_list
df['Kilometraje_del_vehiculo'] = km_car_list
print(df.head(5))

                   Nombre_del_vehiculo Precio_del_vehiculo  \
0      Toyota Fortuner 2.4l Street 4x2         179.000.000   
1                Toyota Hilux 2.7 Wt-i          65.000.000   
2  Mercedes-benz Clase Gla 1.3 Gla 200         170.000.000   
3         Kia Sorento Ex 3.5 Xm At 4x4          59.000.000   
4               Ford Escape 2.0 Se 4x2          56.500.000   

   Modelo_del_vehiculo  Kilometraje_del_vehiculo  
0                 2020                     29032  
1                 2008                    234565  
2                 2022                     14000  
3                 2012                    156712  
4                 2015                     65200  


#### 4.4 Ubicación del carro

In [44]:
#Creamos una lista vacio donde vamos a almacenar la informacion
location_car_list = []

for car in cars:
    location_car = car.find('span', class_ = 'poly-component__location').text
    location_car_list.append(location_car)

df['Ubicacion_del_vehiculo'] = location_car_list
print(df.head(5))

                   Nombre_del_vehiculo Precio_del_vehiculo  \
0      Toyota Fortuner 2.4l Street 4x2         179.000.000   
1                Toyota Hilux 2.7 Wt-i          65.000.000   
2  Mercedes-benz Clase Gla 1.3 Gla 200         170.000.000   
3         Kia Sorento Ex 3.5 Xm At 4x4          59.000.000   
4               Ford Escape 2.0 Se 4x2          56.500.000   

   Modelo_del_vehiculo  Kilometraje_del_vehiculo Ubicacion_del_vehiculo  
0                 2020                     29032     Suba - Bogotá D.C.  
1                 2008                    234565     Suba - Bogotá D.C.  
2                 2022                     14000     Suba - Bogotá D.C.  
3                 2012                    156712     Suba - Bogotá D.C.  
4                 2015                     65200     Suba - Bogotá D.C.  


'https://articulo.tucarro.com.co/MCO-2607872576-toyota-fortuner-24l-street-4x2-_JM#polycard_client=search-nordic&position=1&search_layout=grid&type=item&tracking_id=aec05ad2-c47e-4cc8-b969-e8f6e561e6ce'

In [52]:
#Creamos una lista vacio donde vamos a almacenar la informacion
links_car_list = []

for car in cars:
    link_car = car.find('a', class_ = 'poly-component__title')['href']
    links_car_list.append(link_car)

df['link'] = links_car_list
print(df.head(5))

                   Nombre_del_vehiculo Precio_del_vehiculo  \
0      Toyota Fortuner 2.4l Street 4x2         179.000.000   
1                Toyota Hilux 2.7 Wt-i          65.000.000   
2  Mercedes-benz Clase Gla 1.3 Gla 200         170.000.000   
3         Kia Sorento Ex 3.5 Xm At 4x4          59.000.000   
4               Ford Escape 2.0 Se 4x2          56.500.000   

   Modelo_del_vehiculo  Kilometraje_del_vehiculo Ubicacion_del_vehiculo  \
0                 2020                     29032     Suba - Bogotá D.C.   
1                 2008                    234565     Suba - Bogotá D.C.   
2                 2022                     14000     Suba - Bogotá D.C.   
3                 2012                    156712     Suba - Bogotá D.C.   
4                 2015                     65200     Suba - Bogotá D.C.   

                                                link  
0  https://articulo.tucarro.com.co/MCO-2607872576...  
1  https://articulo.tucarro.com.co/MCO-2607793142...  
2  https://

#### 4.5 Marca del vehiculo

➡️ Aun no sabemos la marca del vehículo, necesitamos extraerla de la columna *'Nombre_del_vehiculo'*

In [53]:
df['Marca_del_vehiculo'] = df['Nombre_del_vehiculo'].str.split(" ").str[0]
print(df.head(5))

                   Nombre_del_vehiculo Precio_del_vehiculo  \
0      Toyota Fortuner 2.4l Street 4x2         179.000.000   
1                Toyota Hilux 2.7 Wt-i          65.000.000   
2  Mercedes-benz Clase Gla 1.3 Gla 200         170.000.000   
3         Kia Sorento Ex 3.5 Xm At 4x4          59.000.000   
4               Ford Escape 2.0 Se 4x2          56.500.000   

   Modelo_del_vehiculo  Kilometraje_del_vehiculo Ubicacion_del_vehiculo  \
0                 2020                     29032     Suba - Bogotá D.C.   
1                 2008                    234565     Suba - Bogotá D.C.   
2                 2022                     14000     Suba - Bogotá D.C.   
3                 2012                    156712     Suba - Bogotá D.C.   
4                 2015                     65200     Suba - Bogotá D.C.   

                                                link Marca_del_vehiculo  
0  https://articulo.tucarro.com.co/MCO-2607872576...             Toyota  
1  https://articulo.tucarro.

#### 4.6 Timestand de la operacion y metadata

➡️ Es importante que siempre haya un registro historico de las operaciones para efectos de trazabilidad. Otra metadata que podriamos agregar es la pagina de la que se esta extrayendo la informacion

In [54]:
df['Registro_pagina_principal'] = pd.Timestamp.now()
df['origen_general'] = conexion_url
df

,Nombre_del_vehiculo,Precio_del_vehiculo,Modelo_del_vehiculo,Kilometraje_del_vehiculo,Ubicacion_del_vehiculo,link,Marca_del_vehiculo,Registro_pagina_principal,origen_general
0,Toyota Fortuner 2.4l Street 4x2,179.000.000,2020,29032,Suba - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-2607872576...,Toyota,2024-08-11 17:10:59.702786,https://carros.tucarro.com.co/
1,Toyota Hilux 2.7 Wt-i,65.000.000,2008,234565,Suba - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-2607793142...,Toyota,2024-08-11 17:10:59.702786,https://carros.tucarro.com.co/
2,Mercedes-benz Clase Gla 1.3 Gla 200,170.000.000,2022,14000,Suba - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-1464945889...,Mercedes-benz,2024-08-11 17:10:59.702786,https://carros.tucarro.com.co/
3,Kia Sorento Ex 3.5 Xm At 4x4,59.000.000,2012,156712,Suba - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-1464887263...,Kia,2024-08-11 17:10:59.702786,https://carros.tucarro.com.co/
4,Ford Escape 2.0 Se 4x2,56.500.000,2015,65200,Suba - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-1464909871...,Ford,2024-08-11 17:10:59.702786,https://carros.tucarro.com.co/
5,Dodge Journey 2.4 Se 4x2,58.900.000,2018,60367,Suba - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-2607358872...,Dodge,2024-08-11 17:10:59.702786,https://carros.tucarro.com.co/
6,Renault Mégane Iii 2.0 Privilege,43.000.000,2013,85000,Teusaquillo - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-2606712694...,Renault,2024-08-11 17:10:59.702786,https://carros.tucarro.com.co/
7,Mazda Cx-5 Grand Touring Lx Tp 2.5,135.000.000,2022,40377,Fontibón - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-2606673842...,Mazda,2024-08-11 17:10:59.702786,https://carros.tucarro.com.co/
8,Renault Stepway Expression 1.6,39.500.000,2017,94000,Kennedy - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-2606737470...,Renault,2024-08-11 17:10:59.702786,https://carros.tucarro.com.co/
9,Mazda 2 1.5 Touring Sedan,58.000.000,2019,68000,Barrios Unidos - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-2606698816...,Mazda,2024-08-11 17:10:59.702786,https://carros.tucarro.com.co/


🗒️ **Observación:** En este punto podemos usar las propiedades de tablas SQL para vincular la informacion general que se está extrayendo en este punto, con la que vamos a extraer posteriormente para cada uno de los items.

Para tal efecto, necesitamos tener una llave que una ambas tablas, y en este caso, la llave es el identficador de cada vehículo que aparece en la url y que inicia con MCO.

In [55]:
def extract_mco_code(url):
    import re
    pattern = r'MCO-\d+'
    match = re.search(pattern, url)
    if match:
        return match.group().replace('-', '')
    else:
        return None

df['Key'] = df['link'].apply(extract_mco_code)
df

,Nombre_del_vehiculo,Precio_del_vehiculo,Modelo_del_vehiculo,Kilometraje_del_vehiculo,Ubicacion_del_vehiculo,link,Marca_del_vehiculo,Registro_pagina_principal,origen_general,Key
0,Toyota Fortuner 2.4l Street 4x2,179.000.000,2020,29032,Suba - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-2607872576...,Toyota,2024-08-11 17:10:59.702786,https://carros.tucarro.com.co/,MCO2607872576
1,Toyota Hilux 2.7 Wt-i,65.000.000,2008,234565,Suba - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-2607793142...,Toyota,2024-08-11 17:10:59.702786,https://carros.tucarro.com.co/,MCO2607793142
2,Mercedes-benz Clase Gla 1.3 Gla 200,170.000.000,2022,14000,Suba - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-1464945889...,Mercedes-benz,2024-08-11 17:10:59.702786,https://carros.tucarro.com.co/,MCO1464945889
3,Kia Sorento Ex 3.5 Xm At 4x4,59.000.000,2012,156712,Suba - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-1464887263...,Kia,2024-08-11 17:10:59.702786,https://carros.tucarro.com.co/,MCO1464887263
4,Ford Escape 2.0 Se 4x2,56.500.000,2015,65200,Suba - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-1464909871...,Ford,2024-08-11 17:10:59.702786,https://carros.tucarro.com.co/,MCO1464909871
5,Dodge Journey 2.4 Se 4x2,58.900.000,2018,60367,Suba - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-2607358872...,Dodge,2024-08-11 17:10:59.702786,https://carros.tucarro.com.co/,MCO2607358872
6,Renault Mégane Iii 2.0 Privilege,43.000.000,2013,85000,Teusaquillo - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-2606712694...,Renault,2024-08-11 17:10:59.702786,https://carros.tucarro.com.co/,MCO2606712694
7,Mazda Cx-5 Grand Touring Lx Tp 2.5,135.000.000,2022,40377,Fontibón - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-2606673842...,Mazda,2024-08-11 17:10:59.702786,https://carros.tucarro.com.co/,MCO2606673842
8,Renault Stepway Expression 1.6,39.500.000,2017,94000,Kennedy - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-2606737470...,Renault,2024-08-11 17:10:59.702786,https://carros.tucarro.com.co/,MCO2606737470
9,Mazda 2 1.5 Touring Sedan,58.000.000,2019,68000,Barrios Unidos - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-2606698816...,Mazda,2024-08-11 17:10:59.702786,https://carros.tucarro.com.co/,MCO2606698816


### **5. Identificamos las páginas que vamos a consultar**

➡️ Para este ejercicio vamos a consultar unicamente las primeras 10 paginas disponibles. Vamos primero a declarar un diccionario que nos ayude a documentar las paginas en las que nos vamos a mover, la inicial termina con sufijo **"/_NoIndex_True"**

In [56]:
paginas = {}

➡️ Podemos observar que cada pagina tiene 48 elementos, y que la segunda pagina arranca en *"49 - _Desde_49_NoIndex_True"*, la tercera en *97 - _Desde_97_NoIndex_True*, es decir con 48 elementos de diferencia... 🤯 **"hay está el patrón"**

In [57]:
for i in range(1,11):
    if i == 1:
        paginas[str(i)] = f'_NoIndex_True'
    else:
        paginas[str(i)] = f'_Desde_{48*(i-1)+1}_NoIndex_True'
    print(paginas)

{'1': '_NoIndex_True'}
{'1': '_NoIndex_True', '2': '_Desde_49_NoIndex_True'}
{'1': '_NoIndex_True', '2': '_Desde_49_NoIndex_True', '3': '_Desde_97_NoIndex_True'}
{'1': '_NoIndex_True', '2': '_Desde_49_NoIndex_True', '3': '_Desde_97_NoIndex_True', '4': '_Desde_145_NoIndex_True'}
{'1': '_NoIndex_True', '2': '_Desde_49_NoIndex_True', '3': '_Desde_97_NoIndex_True', '4': '_Desde_145_NoIndex_True', '5': '_Desde_193_NoIndex_True'}
{'1': '_NoIndex_True', '2': '_Desde_49_NoIndex_True', '3': '_Desde_97_NoIndex_True', '4': '_Desde_145_NoIndex_True', '5': '_Desde_193_NoIndex_True', '6': '_Desde_241_NoIndex_True'}
{'1': '_NoIndex_True', '2': '_Desde_49_NoIndex_True', '3': '_Desde_97_NoIndex_True', '4': '_Desde_145_NoIndex_True', '5': '_Desde_193_NoIndex_True', '6': '_Desde_241_NoIndex_True', '7': '_Desde_289_NoIndex_True'}
{'1': '_NoIndex_True', '2': '_Desde_49_NoIndex_True', '3': '_Desde_97_NoIndex_True', '4': '_Desde_145_NoIndex_True', '5': '_Desde_193_NoIndex_True', '6': '_Desde_241_NoIndex_True